In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [748 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports In

In [7]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-15 23:54:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-05-15 23:54:07 (5.86 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [10]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

## Drop duplicates and incomplete rows

In [11]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

960872
960679
960679


## Examine the schema

In [12]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Rename columns

In [13]:


count_df=df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
count_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12937860|             1|
|   14327548|             4|
|   22158149|             5|
|   12828562|             2|
|    4089580|             1|
|   20097895|             1|
|    2422277|             4|
|   35523976|             2|
|   14051742|             1|
|   14404180|             5|
|   12883856|             3|
|    9942583|             2|
|   38893812|             1|
|   22754948|             1|
|   51265055|             5|
|   49225180|             1|
|   16215726|             1|
|   39948072|            20|
|   14847189|             3|
|   17207294|             2|
+-----------+--------------+
only showing top 20 rows



## Create a new DataFrame for employee info

In [14]:
Review_ID_DF=df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
Review_ID_DF.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OPQYUMAHS|   43765005|B0000TIIZ0|     529990321| 2008-03-05|
|R1010XUX2AIF10|   44007561|B004KTK2M8|      38406392| 2013-04-11|
|R102DONPK6YEGN|   52805367|B00NHEU9U2|      33550162| 2015-06-23|
|R102N48HXFM9WY|   18449245|B000PSQXWG|     517553139| 2013-02-04|
|R1060WX8UX26JV|   18106128|B0045H3K0G|     989815375| 2013-04-15|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [15]:
Product_DF =df.select(["product_id", "product_title"])
Product_DF.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0000TIIZ0|Timex Women's T47...|
|B004KTK2M8|Leather Watchband...|
|B00NHEU9U2|Fanmis Men's Wate...|
|B000PSQXWG|Nautica Men's N14...|
|B0045H3K0G|Columbia Men's CT...|
+----------+--------------------+
only showing top 5 rows



In [16]:
print(df.count())
Product2_df =Product_DF.dropna()
print(Product2_df.count())
Product2_df = Product_DF.dropDuplicates()
print(Product2_df.count())

960679
960679
149694


In [17]:
Customers_DF =df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
Customers_DF.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12937860|             1|
|   14327548|             4|
|   22158149|             5|
|   12828562|             2|
|    4089580|             1|
+-----------+--------------+
only showing top 5 rows



In [18]:
print(Customers_DF.count())
Customer2_df =Customers_DF.dropna()
print(Customer2_df.count())
Customer2_df = Customers_DF.dropDuplicates()
print(Customer2_df.count())

719799
719799
719799


In [19]:
Vine_DF = df.select(["review_id", "star_rating","helpful_votes", "total_votes", "vine" ])
Vine_DF.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OPQYUMAHS|          1|            4|          5|   N|
|R1010XUX2AIF10|          5|            0|          0|   N|
|R102DONPK6YEGN|          5|            0|          0|   N|
|R102N48HXFM9WY|          5|            0|          0|   N|
|R1060WX8UX26JV|          2|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



## Write DataFrame to RDS

In [20]:
print(Vine_DF.count())
Vine2_df =Vine_DF.dropna()
print(Vine2_df.count())
Vine2_df = Vine_DF.dropDuplicates()
print(Vine2_df.count())

960679
960679
960679


In [21]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-2.cb1bticre0py.us-east-1.rds.amazonaws.com:5432/Big_Data"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

## Create a new DataFrame for employee passwords

## Write DataFrame to RDS

In [23]:
# Write DataFrame to table

Review_ID_DF.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.2.1-bin-hadoop2.7/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.2.1-bin-hadoop2.7/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
Product2_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
Vine2_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
Customer2_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)